In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(
  temperature=0.1,
)

cache_dir = LocalFileStore("./cache/")



splitter = CharacterTextSplitter.from_tiktoken_encoder(
  separator="\n",
  chunk_size=600,
  chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)


embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
  embeddings, cache_dir
)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

retrieval = vectorstore.as_retriever()

ptrompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant. Answer questions using only the context. If you don't know the answer, just say you don't know; don't make it up.:\n{context}"),
  ("human", "{question}")])

chain = (
  {
    "context": retrieval,
    "question": RunnablePassthrough(), 
    "extra:": RunnablePassthrough()
  }
  | ptrompt
  | llm
)

chain.invoke("Describe Victory Mansions")

ImportError: cannot import name 'PydanticDeprecationWarning' from 'pydantic' (c:\Users\bush\Documents\GitHub\fullstack-gpt\env\Lib\site-packages\pydantic\__init__.cp311-win_amd64.pyd)

In [6]:
results = vectorstore.similarity_search("where does winston live")

print(results)

[Document(page_content='The Ministry of Love was the really frightening one. There were no windows in it at all. Winston had never been inside the Ministry of Love, nor within half a kilometre of it. It was a place impossible to enter except on official business, and then only by penetrating through a maze of barbed-wire entanglements, steel doors, and hidden machine-gun nests. Even the streets leading up to its outer barriers were roamed by gorilla-faced guards in black uniforms, armed with jointed truncheons.\nWinston turned round abruptly. He had set his features into the expression of quiet optimism which it was advisable to wear when facing the telescreen. He crossed the room into the tiny kitchen. By leaving the Ministry at this time of day he had sacrificed his lunch in the canteen, and he was aware that there was no food in the kitchen except a hunk of dark-coloured bread which had got to be saved for tomorrow’s breakfast. He took down from the shelf a bottle of colourless liqu